In [1]:
import os
import math
import torch
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np
import networkx as nx
import GraphTransforms as Tr
import torch_geometric.transforms as T
import GraphNN as G
import GraphDataSets as D
import random

from tqdm.notebook import tqdm
from sklearn.metrics import roc_curve, auc
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_networkx
from mpl_toolkits.axes_grid1 import make_axes_locatable

/lhome/ific/f/fkellere/miniconda/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
dataset_name = 'RecoBig_all_10mm_R2'
transform    = True
DS = getattr(D,dataset_name)

In [3]:
if transform:
    dataset = DS(root='./GNN_datasets/',transform=Tr.RandomNodeSplit())#, pre_transform=transform)
    dataset_name += '_T'
else:
    dataset   = DS(root='GNN_datasets/')

In [4]:
%mkdir 'IN_{dataset_name}'
train_dataset = dataset[:int(1/2*len(dataset))]
valid_dataset = dataset[int(1/2*len(dataset)):int(3/4*len(dataset))]
test_dataset  = dataset[int(3/4*len(dataset)):]

mkdir: cannot create directory ‘IN_RecoBig_all_10mm_R2_T’: File exists


In [5]:
path  = dataset_name+'_best.pth'
model = G.InteractionNetwork(128, dataset=dataset, inputs = dataset.num_node_features, outputs = 2)
model.load_state_dict(torch.load(path,map_location='cpu'))
model.eval()

InteractionNetwork(
  (interactionnetwork): MetaLayer(
    edge_model=EdgeBlock(
    (edge_mlp): Sequential(
      (0): Linear(in_features=8, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=128, bias=True)
    )
  ),
    node_model=NodeBlock(
    (node_mlp_1): Sequential(
      (0): Linear(in_features=132, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=128, bias=True)
    )
    (node_mlp_2): Sequential(
      (0): Linear(in_features=132, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=128, bias=True)
    )
  ),
    global_model=GlobalBlock(
    (global_mlp): Sequential(
     

In [6]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

In [7]:
def IN_analysis(loader):
    t = tqdm(enumerate(loader),total=len(loader))
    y_test = []
    y_predict = []
    for i,data in t: 
        try:
            batch_output = model(data.x, data.edge_index, data.edge_attr, data.u, data.batch)
        except:
            batch_output = model(data.x, data.edge_index, data.edge_attr, None, data.batch)
        for j,obj in enumerate(batch_output.detach().cpu().numpy()):
            y_predict.append(batch_output.detach().cpu().numpy()[j][1])
        y_test.append(data.y.cpu().numpy())
    y_test = np.concatenate(y_test)
    y_predict = np.array(y_predict)
    y_predict = np.exp(y_predict)/(1+np.exp(y_predict))
        
    #calculate scores
    B_score = []
    S_score = []
    t = tqdm(range(0, len(y_predict)))

    for i in t:
        if y_test[i]==0:
            B_score.append(y_predict[i])
        else:
            S_score.append(y_predict[i])
            
    # create ROC curves and FOM
    fpr_gnn, tpr_gnn, threshold_gnn = roc_curve(y_test, y_predict)
    Seff = []
    Brej = []
    Figm = []
    thre = np.linspace(min(y_predict),max(y_predict),100)
    t    = tqdm(range(0,len(thre)))

    for i in t:
        Seff.append(len([x for x in S_score if x>=thre[i]])/len(S_score))
        Brej.append(1-len([x for x in B_score if x>=thre[i]])/len(B_score))
        Figm.append(len([x for x in S_score if x>=thre[i]])*np.sqrt(len(B_score))/
                (np.sqrt(len([x for x in B_score if x>=thre[i]]))*len(S_score)))
        

    return y_test, y_predict, fpr_gnn, tpr_gnn, threshold_gnn, Seff, Brej, Figm, thre, B_score, S_score

In [ ]:
y_test, y_predict, fpr_gnn_tr, tpr_gnn_tr, threshold_gnn_tr, Seff_tr, Brej_tr, Figm_tr, thre_tr, B_score_tr, S_score_tr = IN_analysis(train_loader)
y_test, y_predict, fpr_gnn_vl, tpr_gnn_vl, threshold_gnn_vl, Seff_vl, Brej_vl, Figm_vl, thre_vl, B_score_vl, S_score_vl = IN_analysis(valid_loader)
y_test, y_predict, fpr_gnn_te, tpr_gnn_te, threshold_gnn_te, Seff_te, Brej_te, Figm_te, thre_te, B_score_te, S_score_te = IN_analysis( test_loader)


  0%|          | 0/4740 [00:00<?, ?it/s]

In [ ]:
#plt.style.use(hep.style.ROOT)

store = pd.HDFStore("Loss_"+dataset_name+".h5")
plt.plot(store['loss_tr'],label='Training')
plt.plot(store['loss_te'],label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
#plt.ylim([0,1])
plt.legend()
plt.savefig(f'IN_{dataset_name}/Loss.png',bbox_inches='tight')

In [ ]:
plt.style.use(hep.style.ROOT)
    
# plot ROC curves
plt.figure()
plt.plot(tpr_gnn_tr, fpr_gnn_tr, lw=2.5, label="Training, AUC = {:.1f}%".format(auc(fpr_gnn_tr,tpr_gnn_tr)*100))
plt.plot(tpr_gnn_vl, fpr_gnn_vl, lw=2.5, label="Validation, AUC = {:.1f}%".format(auc(fpr_gnn_vl,tpr_gnn_vl)*100))
plt.plot(tpr_gnn_te, fpr_gnn_te, lw=2.5, label="Test, AUC = {:.1f}%".format(auc(fpr_gnn_te,tpr_gnn_te)*100))
plt.xlabel(r'True positive rate')
plt.ylabel(r'False positive rate')
plt.semilogy()
plt.ylim(0.001, 1)
plt.xlim(0, 1)
plt.grid(True)
plt.legend(loc='upper left')
plt.savefig(f'IN_{dataset_name}/ROC.png',bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use(hep.style.ROOT)

plt.hist(S_score_te, 50, label='Signal score', alpha=0.7)
plt.hist(B_score_te, 50, label='Background score', alpha=0.7)
#plt.semilogy()
plt.legend()
plt.savefig(f'IN_{dataset_name}/Scoring.png',bbox_inches='tight')

In [ ]:
plt.style.use(hep.style.ROOT)

plt.figure()
plt.plot(Brej_tr, Seff_tr, lw=2.5, label="Training, AUC = {:.1f}%".format(auc(Brej_tr,Seff_tr)*100))
plt.plot(Brej_vl, Seff_vl, lw=2.5, label="Validation, AUC = {:.1f}%".format(auc(Brej_vl,Seff_vl)*100))
plt.plot(Brej_te, Seff_te, lw=2.5, label="Test, AUC = {:.1f}%".format(auc(Brej_te,Seff_te)*100))
plt.xlabel(r'Background Rejection')
plt.ylabel(r'Signal Efficiency')
plt.ylim(0.001, 1)
plt.xlim(0.001, 1)
plt.grid(True)
plt.legend(loc='upper left')
plt.savefig(f'IN_{dataset_name}/ROC2.png',bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use(hep.style.ROOT)

plt.figure()
plt.plot(thre_tr, Figm_tr, lw=2.5, label = "Training")
plt.plot(thre_vl, Figm_vl, lw=2.5, label = "Validation")
plt.plot(thre_te, Figm_te, lw=2.5, label = "Test")
plt.xlabel(r'GNN Prediction Threshold')
plt.ylabel(r'f.o.m. ($\epsilon_{s}/\sqrt{\epsilon_{b}}$)')
plt.grid(True)
plt.legend()
plt.savefig(f'IN_{dataset_name}/FOM.png',bbox_inches='tight')
plt.show()

In [ ]:
fom  = [x for x in Figm_te if x!=np.inf]
xmax = thre_te[np.argmax(fom)]

In [ ]:
X = []
Y = []
Z = []
t = tqdm(range(0, len(test_dataset)))
for i in t:
    X.append(np.mean(test_dataset[i].u.numpy().transpose()[0]))
    Y.append(np.mean(test_dataset[i].u.numpy().transpose()[1]))
    Z.append(np.mean(test_dataset[i].u.numpy().transpose()[2]))

In [ ]:
X_sorted, keys_x = zip(*sorted(zip(X,range(0,len(X)))))
Y_sorted, keys_y = zip(*sorted(zip(Y,range(0,len(Y)))))
Z_sorted, keys_z = zip(*sorted(zip(Z,range(0,len(Z)))))

In [ ]:
nBins = 6
Xbins = np.linspace(min(X),max(X),nBins+1)
Ybins = np.linspace(min(Y),max(Y),nBins+1)
Zbins = np.linspace(min(Z),max(Z),nBins+1)
XYauc = []
XZauc = []
YZauc = []
XYRS  = []
XZRS  = []
YZRS  = []
XYRB  = []
XZRB  = []
YZRB  = []
s     = tqdm(range(0,nBins))

for i in s:
    keys_xi = keys_x[len([x for x in X if x<=Xbins[i]]):len([x for x in X if x<=Xbins[i+1]])]
    keys_yi = keys_y[len([x for x in Y if x<=Ybins[i]]):len([x for x in Y if x<=Ybins[i+1]])]
    keys_zi = keys_z[len([x for x in Z if x<=Zbins[i]]):len([x for x in Z if x<=Zbins[i+1]])]
    for j in tqdm(range(0,nBins)):
        keys_xj = keys_x[len([x for x in X if x<=Xbins[j]]):len([x for x in X if x<=Xbins[j+1]])]
        keys_yj = keys_y[len([x for x in Y if x<=Ybins[j]]):len([x for x in Y if x<=Ybins[j+1]])]
        keys_zj = keys_z[len([x for x in Z if x<=Zbins[j]]):len([x for x in Z if x<=Zbins[j+1]])]
        keys_xy = list(set(keys_xi).intersection(keys_yj))
        keys_xz = list(set(keys_xi).intersection(keys_zj))
        keys_yz = list(set(keys_yi).intersection(keys_zj))
        a       = np.array([y_test[x] for x in keys_xy])
        b       = np.array([y_predict[x] for x in keys_xy])
        PredS   = np.array([a[i] for i in range(0,len(a)) if b[i]>=xmax])
        PredB   = np.array([a[i] for i in range(0,len(a)) if b[i]<xmax])
        try:
            XYRS.append(len(PredS[PredS==1])/len(a[a==1]))
        except ZeroDivisionError:
            XYRS.append(0)
        try:
            XYRB.append(len(PredB[PredB==0])/len(a[a==0]))
        except ZeroDivisionError:
            XYRB.append(0)
        fpr, tpr, threshold = roc_curve(a,b)
        XYauc.append(auc(fpr,tpr))
        a    = np.array([y_test[x] for x in keys_xz])
        b    = np.array([y_predict[x] for x in keys_xz])
        PredS= np.array([a[i] for i in range(0,len(a)) if b[i]>=xmax])
        PredB= np.array([a[i] for i in range(0,len(a)) if b[i]<xmax])
        try:
            XZRS.append(len(PredS[PredS==1])/len(a[a==1]))
        except ZeroDivisionError:
            XZRS.append(0)
        try:
            XZRB.append(len(PredB[PredB==0])/len(a[a==0]))
        except ZeroDivisionError:
            XZRB.append(0)
        fpr, tpr, threshold = roc_curve(a,b)
        XZauc.append(auc(fpr,tpr))
        a    = np.array([y_test[x] for x in keys_yz])
        b    = np.array([y_predict[x] for x in keys_yz])
        PredS= np.array([a[i] for i in range(0,len(a)) if b[i]>=xmax])
        PredB= np.array([a[i] for i in range(0,len(a)) if b[i]<xmax])
        try:
            YZRS.append(len(PredS[PredS==1])/len(a[a==1]))
        except ZeroDivisionError:
            XYRS.append(0)
        try:
            YZRB.append(len(PredB[PredB==0])/len(a[a==0]))
        except ZeroDivisionError:
            YZRB.append(0)
        fpr, tpr, threshold = roc_curve(a,b)
        YZauc.append(auc(fpr,tpr))
        
xy   = np.flipud(np.array(XYauc).reshape(nBins,nBins).transpose())
xz   = np.flipud(np.array(XZauc).reshape(nBins,nBins).transpose())
yz   = np.flipud(np.array(YZauc).reshape(nBins,nBins).transpose())
xyrs = np.flipud(np.array(XYRS).reshape(nBins,nBins).transpose())
xzrs = np.flipud(np.array(XZRS).reshape(nBins,nBins).transpose())
yzrs = np.flipud(np.array(YZRS).reshape(nBins,nBins).transpose())
xyrb = np.flipud(np.array(XYRB).reshape(nBins,nBins).transpose())
xzrb = np.flipud(np.array(XZRB).reshape(nBins,nBins).transpose())
yzrb = np.flipud(np.array(YZRB).reshape(nBins,nBins).transpose())

In [ ]:
plt.rc('axes',  titlesize=12)
plt.rc('axes',  labelsize=16)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(12, 6))
im1 = ax0.imshow(xy,extent=(0,max(X),0,max(Y)))#,vmin=0.94,vmax=0.965)
ax0.set(xlabel='X',ylabel='Y')
divider = make_axes_locatable(ax0)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im1, cax=cax,format='%.2f')
im2 = ax1.imshow(xz,extent=(0,max(X),0,max(Z)))#,vmin=0.94,vmax=0.965)
ax1.set(xlabel='X',ylabel='Z')
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im2, cax=cax,format='%.2f')
im3 = ax2.imshow(yz,extent=(0,max(Y),0,max(Z)))#,vmin=0.94,vmax=0.965)
ax2.set(xlabel='Y',ylabel='Z')
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im3, cax=cax,format='%.2f')
fig.suptitle('AUC score spatial distribution',fontsize=20)
fig.tight_layout()
plt.savefig(f'IN_{dataset_name}/AUC_dist.png',bbox_inches='tight')

In [ ]:
Xsig = []
Ysig = []
Zsig = []
Xbkg = []
Ybkg = []
Zbkg = []
t = tqdm(range(0, len(test_dataset)))
for i in t:
    if test_dataset[i].y==0:
        Xbkg.append(np.mean(test_dataset[i].u.numpy().transpose()[0]))
        Ybkg.append(np.mean(test_dataset[i].u.numpy().transpose()[1]))
        Zbkg.append(np.mean(test_dataset[i].u.numpy().transpose()[2]))
    else:
        Xsig.append(np.mean(test_dataset[i].u.numpy().transpose()[0]))
        Ysig.append(np.mean(test_dataset[i].u.numpy().transpose()[1]))
        Zsig.append(np.mean(test_dataset[i].u.numpy().transpose()[2]))

In [ ]:
plt.rcParams['figure.constrained_layout.use'] = True
plt.rc('axes',  titlesize=16)
plt.rc('axes',  labelsize=16)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(12, 6))
im1 = ax0.hist2d(Xsig,Ysig)
ax0.set(xlabel='X',ylabel='Y',title='Signal Events')
fig.colorbar(im1[3],ax=ax0)
im2 = ax1.hist2d(Xbkg,Ybkg)
ax1.set(xlabel='X',ylabel='Y',title='Background Events')
fig.colorbar(im2[3],ax=ax1)
plt.savefig(f'IN_{dataset_name}/EventDist.png',bbox_inches='tight')

In [ ]:
# create energy histograms
#if dataset_name=='DataMCmix' or dataset_name=='DataMCmix_SB50':
#    Data   = pd.read_hdf('./Input_Dataframes/cdst_voxel_Data_calib.h5')
#    MC     = pd.read_hdf('./Input_Dataframes/cdst_voxel_RecoBig.h5')
#    Data   = Data[Data.dataset_id<Data.dataset_id.unique()[-1]]
#    if dataset_name=='DataMCmix_SB50':
#        MC = MC[MC.dataset_id<=MC.dataset_id.unique()[len(Data.dataset_id.unique())+100]]
#    Dindex = np.where(dataset.y==1)[0]
#    Mindex = np.where(dataset.y==0)[0]
#    Dict_D = dict(zip(Data.dataset_id.unique(), Dindex))
#    Dict_M = dict(zip(MC.dataset_id.unique(), Mindex))
#    
#    Data.binclass = 1
#    MC.binclass   = 0
#    Data['dataset_id'] = Data['dataset_id'].replace(Dict_D)
#    eventInfo = MC[['dataset_id']].drop_duplicates().reset_index(drop=True)
#    eventInfo['new_dataset_id'] = eventInfo['dataset_id'].replace(Dict_M)
#    
#    MC     = pd.merge(eventInfo,MC,on='dataset_id',how='right')
#    MC     = MC.drop('dataset_id',axis=1)
#    MC     = MC.rename(columns={'new_dataset_id':'dataset_id'})
#    source = pd.concat([Data,MC])
#    source = source.sort_values(by='dataset_id').reset_index(drop=True)
#else:
# Load source
Raw_Files = DS.raw_file_names.fget(DS)
if len(Raw_Files)==1:
    source = pd.read_hdf(DS.raw_file_names.fget(DS)[0])
else:
    source = pd.DataFrame()
    cntr   = 0
    for file in tqdm(Raw_Files):
        MCpart   = pd.read_hdf(file)

        length = max(MCpart.dataset_id.unique())+1
        MCpart.dataset_id += cntr
        cntr  += length
        source = pd.concat([source,MCpart])
    #create new unique identifier
    eventInfo = source[['dataset_id', 'binclass']].drop_duplicates().reset_index(drop=True)
    dct_map = {eventInfo.iloc[i].dataset_id : i for i in range(len(eventInfo))}
    #add dataset_id, pathname and basename to eventInfo
    eventInfo = eventInfo.assign(dataset_id = eventInfo.dataset_id.map(dct_map))
    #add dataset_id to data and drop event_id
    source = source.assign(dataset_id = source.dataset_id.map(dct_map))
    
source_test = source[source.dataset_id>=source.dataset_id.unique()[int(3/4*len(source.dataset_id.unique()))]]
E           = np.array(source_test.groupby(['dataset_id'])['energy'].sum())

In [ ]:
#Verification of the energy histogram using 100 random samples

R = random.sample(range(0,len(test_dataset)),100)
for i in tqdm(R):
    if (torch.tensor(DS.construct_center(DS,source_test[source_test.dataset_id==source_test.dataset_id.unique()[i]]))!=test_dataset[i].u).any():
        print(i)
        raise ValueError(f'Source and graph dataset indices are not matching up at index {i}!')

In [ ]:
NBins = 20
EBins = np.linspace(min(E),max(E),NBins+1)
E_sorted, keys_e = zip(*sorted(zip(E,range(0,len(y_test)))))
ERS   = []
ERB   = []
ERSe  = []
ERBe  = []

for i in tqdm(range(0,NBins)):
    keys_ei = keys_e[len([x for x in E if x<=EBins[i]]):len([x for x in E if x<=EBins[i+1]])]
    a       = np.array([y_test[x]    for x in keys_ei])
    b       = np.array([y_predict[x] for x in keys_ei])
    PredS   = np.array([a[i] for i in range(0,len(a)) if b[i]>=0.5])
    PredB   = np.array([a[i] for i in range(0,len(a)) if b[i]<0.5])
    Seff    = len(PredS[PredS==1])/len(a[a==1])
    Beff    = len(PredB[PredB==0])/len(a[a==0])
    ERS.append(Seff)
    ERB.append(Beff)
    ERSe.append(Seff*np.sqrt(1/len(PredS[PredS==1])+1/len(a[a==1])))
    ERBe.append(Beff*np.sqrt(1/len(PredB[PredB==0])+1/len(a[a==0])))

In [ ]:
plt.style.use(hep.style.ROOT)

plt.errorbar(EBins[1:],ERS,ERSe,label='Signal')
plt.errorbar(EBins[1:],ERB,ERBe,label='Background')
plt.ylabel('Selection Efficiency')
plt.xlabel('Energy (MeV)')
plt.legend()
plt.savefig(f'IN_{dataset_name}/Efficiency.png',bbox_inches='tight')

In [ ]:
S           = []
B           = []
for i in tqdm(range(0,len(E))):
    if y_predict[i]<xmax:
        B.append(E[i])
    else:
        S.append(E[i])

In [ ]:
plt.hist(E,50,label='All events')
plt.hist(S,50,histtype='step',label='Labeled signal')
plt.hist(B,50,histtype='step',label='Labeled background');
plt.xlabel('MeV')
plt.legend()

In [ ]:
plt.rc('axes',  titlesize=12)
plt.rc('axes',  labelsize=16)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(12, 6))
im1 = ax0.imshow(xyrs,extent=(0,max(X),0,max(Y)))#,vmin=0.94,vmax=0.965)
ax0.set(xlabel='X',ylabel='Y')
divider = make_axes_locatable(ax0)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im1, cax=cax,format='%.2f')
im2 = ax1.imshow(xzrs,extent=(0,max(X),0,max(Z)))#,vmin=0.94,vmax=0.965)
ax1.set(xlabel='X',ylabel='Z')
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im2, cax=cax,format='%.2f')
im3 = ax2.imshow(yzrs,extent=(0,max(Y),0,max(Z)))#,vmin=0.94,vmax=0.965)
ax2.set(xlabel='Y',ylabel='Z')
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im3, cax=cax,format='%.2f')
fig.tight_layout()
plt.savefig(f'IN_{dataset_name}/Signal_eff_dist.png',bbox_inches='tight')

In [ ]:
plt.rc('axes',  titlesize=12)
plt.rc('axes',  labelsize=16)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(12, 6))
im1 = ax0.imshow(xyrb,extent=(0,max(X),0,max(Y)))#,vmin=0.94,vmax=0.965)
ax0.set(xlabel='X',ylabel='Y')
divider = make_axes_locatable(ax0)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im1, cax=cax,format='%.2f')
im2 = ax1.imshow(xzrb,extent=(0,max(X),0,max(Z)))#,vmin=0.94,vmax=0.965)
ax1.set(xlabel='X',ylabel='Z')
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im2, cax=cax,format='%.2f')
im3 = ax2.imshow(yzrb,extent=(0,max(Y),0,max(Z)))#,vmin=0.94,vmax=0.965)
ax2.set(xlabel='Y',ylabel='Z')
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im3, cax=cax,format='%.2f')
fig.tight_layout()
plt.savefig(f'IN_{dataset_name}/Background_eff_dist.png',bbox_inches='tight')

In [ ]:
Seff_te[np.argmax(fom)]

In [ ]:
Brej_te[np.argmax(fom)]

In [ ]:
max(fom)

In [ ]:
thre_IN = thre_te
Figm_IN = Figm_te
Seff_IN = Seff_te
Brej_IN = Brej_te
%store thre_IN
%store Figm_IN
%store Seff_IN
%store Brej_IN

In [ ]:
from iminuit import Minuit, describe
from probfit import Extended, BinnedChi2
from probfit import gaussian, linear, poly2, Chi2Regression
from scipy   import integrate

In [ ]:
def gaussNorm(x, mu, sigma, N):
    return (N*np.exp(-0.5 * np.power((x - mu)/(sigma),2 ))/( sigma * np.sqrt( 2 * np.pi)))

def LorentzNorm(x, mu, sigma, N):
    return N/np.pi*(sigma/((x-mu)**2+sigma**2))

def ShapedGauss(x, mu, sigma, beta, N):
    return N*beta/(4*sigma*math.gamma(1/beta))*np.exp(-0.5*np.power((np.abs(x-mu)/(2*sigma)),beta))

def expNorm(x, C, s, xmin, xmax):
    return C * s * np.exp(-x * s) /  (np.exp(-s * xmin) - np.exp(-s * xmax))

def gaussExp_Norm(x, mu, sigma, N, C, s, xmin, xmax) :
    return gaussNorm(x, mu, sigma, N) + expNorm(x, C, s, xmin, xmax)

def lorentzExp_Norm(x, mu, sigma, N, C, s, xmin, xmax):
    return LorentzNorm(x, mu, sigma, N) + expNorm(x, C, s, xmin, xmax)

def shapedgaussExp_Norm(x, mu, sigma, beta, N, C, s, xmin, xmax):
    return ShapedGauss(x, mu, sigma, beta, N) + expNorm(x, C, s, xmin, xmax)

In [ ]:
# Create the cost function.
bins_fit   = 50
fit_range  = (1.5, 1.7)
mu         = 1.6
sigma      = 0.007
N          = 40000
C          = 50000
s          = 1.5

#plt.figure(figsize=(5,4))
chi2 = BinnedChi2(gaussExp_Norm, E, bins = bins_fit , bound=fit_range)
chi2.show(args={'mu':mu, 'sigma':sigma, 'N':N,  'C':C, 's':s , 'xmin':fit_range[0], 'xmax':fit_range[1]})
plt.show()

In [ ]:
# Perform the fit.
#plt.figure(figsize=(8,5))
minuit = Minuit(chi2,mu=mu,sigma=sigma,N=N, C=C,s=s,xmin=fit_range[0],xmax=fit_range[1],fix_xmin=True,fix_xmax=True)
minuit.migrad()
chi2.show(minuit)
plt.show()

mean      = minuit.values[0]
mean_u    = minuit.errors[0]

sigma     = minuit.values[1]
sigma_u   = minuit.errors[1]

N         = minuit.values[2]
N_u       = minuit.errors[2]

C         = minuit.values[3]
C_u       = minuit.errors[3]

s         = minuit.values[4]
s_u       = minuit.errors[4]

chi2_result = minuit.fval/chi2.ndof

print(f'Mean:  {mean:.2f}         +/- {mean_u:.5f} ')
print(f'Sigma: {sigma:.2f}        +/- {sigma_u:.5f} ')
print(f'N:     {N:.1f}            +/- {N_u:.0f} ')
print(f'C:     {C:.1f}            +/- {C_u:.0f} ')
print(f's:     {s:.1f}            +/- {s_u:.2f} ')
print(f'chi2:  {chi2_result:.2f}     ')

In [ ]:
Ecut  = E[E>minuit.values[0]-3*minuit.values[1]]
ypcut = y_predict[E>minuit.values[0]-3*minuit.values[1]]
ytcut = y_test[E>minuit.values[0]-3*minuit.values[1]]
ytcut = ytcut[Ecut<minuit.values[0]+3*minuit.values[1]]
ypcut = ypcut[Ecut<minuit.values[0]+3*minuit.values[1]]
Ecut  = Ecut[Ecut<minuit.values[0]+3*minuit.values[1]]

In [ ]:
NB = integrate.quad(lambda x: expNorm(x,minuit.values[3],minuit.values[4],fit_range[0],fit_range[1]),
                    minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]
NS = integrate.quad(lambda x: gaussNorm(x,minuit.values[0],minuit.values[1],minuit.values[2]),
                    minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]

In [ ]:
ROCx1 = []
ROCy1 = []
FOM1  = []
for c in np.linspace(0,1,20):
    S    = [E[i] for i in range(len(y_predict)) if y_predict[i]>c]
    try:
        chi2 = BinnedChi2(gaussExp_Norm, S, bins = bins_fit , bound=fit_range)
    except ValueError:
        ROCx1.append(1)
        ROCy1.append(0)
        FOM1. append(0)
        continue
    minuit = Minuit(chi2, mu=mu, sigma=sigma, N=N*len(S)/len(E),  C=C*len(S)/len(E),
                    s=s, xmin=fit_range[0], xmax=fit_range[1], fix_xmin=True, fix_xmax=True)
    minuit.migrad()
    chi2_result = minuit.fval/chi2.ndof
    print(f'chi2: {chi2_result}')
    nb = integrate.quad(lambda x: expNorm(x,minuit.values[3],minuit.values[4],minuit.values[5],minuit.values[6]),
                    minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]
    ns = integrate.quad(lambda x: gaussNorm(x,minuit.values[0],minuit.values[1],minuit.values[2]),
                    minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]
    ROCx1.append(1-nb/NB)
    ROCy1.append(ns/NS)
    FOM1. append(ns*np.sqrt(NB)/(NS*np.sqrt(nb)))

In [ ]:
fpr_gnn, tpr_gnn, threshold_gnn = roc_curve(ytcut, ypcut)

In [ ]:
plt.plot(ROCx1,ROCy1,label="Fit AUC = {:.1f}%".format(integrate.trapezoid(ROCy1,ROCx1)*100))
plt.plot(Brej_te, Seff_te, lw=2.5, label="Truth AUC = {:.1f}%".format(auc(Brej_te,Seff_te)*100))
plt.plot(1-fpr_gnn, tpr_gnn, lw=2.5, label="Truth AUC in peak region = {:.1f}%".format(auc(fpr_gnn,tpr_gnn)*100))
plt.xlabel(r'Background Rejection')
plt.ylabel(r'Signal Efficiency')
plt.legend()

In [ ]:
S    = [E[i] for i in range(len(y_predict)) if y_predict[i]>5/9]

In [ ]:
chi2 = BinnedChi2(gaussExp_Norm, S, bins = bins_fit , bound=fit_range)

minuit = Minuit(chi2, mu=mu, sigma=sigma, N=N*len(S)/len(E),  C=C*len(S)/len(E),
                    s=s, xmin=fit_range[0], xmax=fit_range[1], fix_xmin=True, fix_xmax=True)
minuit.migrad()

In [ ]:
nb = integrate.quad(lambda x: expNorm(x,minuit.values[3],minuit.values[4],minuit.values[5],minuit.values[6]),
                minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]
ns = integrate.quad(lambda x: gaussNorm(x,minuit.values[0],minuit.values[1],minuit.values[2]),
                minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]

In [ ]:
nb

In [ ]:
ns

In [ ]:
Scut    = Ecut[ypcut>5/9]

In [ ]:
ytscut = ytcut[ypcut>5/9]

In [ ]:
ypscut = ypcut[ypcut>5/9]

In [ ]:
len(ytscut)

In [ ]:
len(Scut)

In [ ]:
nsT = len(Scut[ytscut==1])

In [ ]:
nbT = len(ytscut[ytscut==0])

In [ ]:
NST = len(Ecut[ytcut==1])
NBT = len(Ecut[ytcut==0])

In [ ]:
1-nb/NB

In [ ]:
ns/NS

In [ ]:
1-nbT/NBT

In [ ]:
nsT/NST

In [ ]:
plt.hist(Scut,50);
plt.hist(Scut[ytscut==1],50,histtype='step',lw=2.5);
a = plt.hist(Scut[ytscut==0],50,histtype='step',lw=2.5);
plt.plot(np.linspace(min(Scut),max(Scut),50),gaussNorm(np.linspace(min(Scut),max(Scut),50),
                                                       minuit.values[0],minuit.values[1],minuit.values[2]*
                                                       (max(Scut)-min(Scut))/50))
plt.plot(np.linspace(min(Scut),max(Scut),50),expNorm(np.linspace(min(Scut),max(Scut),50),
                                                       minuit.values[3]*(max(Scut)-min(Scut))/50,
                                                     minuit.values[4],fit_range[0],fit_range[1]))
plt.plot(np.linspace(min(Scut),max(Scut),50),gaussExp_Norm(np.linspace(min(Scut),max(Scut),50),
                                                       minuit.values[0],minuit.values[1],
                                                           minuit.values[2]*(max(Scut)-min(Scut))/50,
                                                           minuit.values[3]*(max(Scut)-min(Scut))/50,
                                                          minuit.values[4],minuit.values[5],minuit.values[6]))

In [ ]:
sum(a[0])

In [ ]:
(NST-NS)/NST

In [ ]:
(NBT-NB)/NBT

In [ ]:
(nsT-ns)/nsT

In [ ]:
(nbT-nb)/nbT

In [ ]:
nb

In [ ]:
nbT

In [ ]:
NB

In [ ]:
NBT

In [ ]:
integrate.quad(lambda x: expNorm(x,minuit.values[3],minuit.values[4],minuit.values[5],minuit.values[6]),
                minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]

In [ ]:
minuit.values[5]

In [ ]:
fit_range

In [ ]:
ROCx1

In [ ]:
ROCy1

In [ ]:
# Create the cost function.
bins_fit   = 50
fit_range  = (1.5, 1.7)
mu         = 1.6
sigma      = 0.0035
beta       = 2
N          = 40000
C          = 50000
s          = 1.5

#plt.figure(figsize=(5,4))
chi2 = BinnedChi2(shapedgaussExp_Norm, E, bins = bins_fit , bound=fit_range)
chi2.show(args={'mu':mu, 'sigma':sigma, 'beta':beta, 'N':N,  'C':C, 's':s , 'xmin':fit_range[0], 'xmax':fit_range[1]})
plt.show()

In [ ]:
# Perform the fit.
#plt.figure(figsize=(8,5))
minuit = Minuit(chi2,mu=mu,sigma=sigma,beta=beta,N=N,C=C,s=s,xmin=fit_range[0],xmax=fit_range[1],fix_xmin=True,
                fix_xmax=True)
minuit.migrad()
chi2.show(minuit)
plt.show()

mean      = minuit.values[0]
mean_u    = minuit.errors[0]

sigma     = minuit.values[1]
sigma_u   = minuit.errors[1]

N         = minuit.values[3]
N_u       = minuit.errors[3]

C         = minuit.values[4]
C_u       = minuit.errors[4]

s         = minuit.values[5]
s_u       = minuit.errors[5]

chi2_result = minuit.fval/chi2.ndof

print(f'Mean:  {mean:.2f}         +/- {mean_u:.5f} ')
print(f'Sigma: {sigma:.2f}        +/- {sigma_u:.5f} ')
print(f'N:     {N:.1f}            +/- {N_u:.0f} ')
print(f'C:     {C:.1f}            +/- {C_u:.0f} ')
print(f's:     {s:.1f}            +/- {s_u:.2f} ')
print(f'chi2:  {chi2_result:.2f}     ')

In [ ]:
NB = integrate.quad(lambda x: expNorm(x,minuit.values[4],minuit.values[5],fit_range[0],fit_range[1]),
                    minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]
NS = integrate.quad(lambda x: ShapedGauss(x,minuit.values[0],minuit.values[1],minuit.values[2],minuit.values[3]),
                    minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]

ROCx2 = []
ROCy2 = []
FOM2  = []
for c in np.linspace(0,1,20):
    S    = [E[i] for i in range(len(y_predict)) if y_predict[i]>c]
    try:
        chi2 = BinnedChi2(shapedgaussExp_Norm, S, bins = bins_fit , bound=fit_range)
    except ValueError:
        ROCx2.append(1)
        ROCy2.append(0)
        FOM2. append(0)
        continue
    minuit = Minuit(chi2, mu=mu, sigma=sigma, beta=beta, N=N*len(S)/len(E),  C=C*len(S)/len(E),
                    s=s, xmin=fit_range[0], xmax=fit_range[1], fix_xmin=True, fix_xmax=True)
    minuit.migrad()
    chi2_result = minuit.fval/chi2.ndof
    print(f'chi2: {chi2_result}')
    nb = integrate.quad(lambda x: expNorm(x,minuit.values[4],minuit.values[5],minuit.values[6],minuit.values[7]),
                    minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]
    ns = integrate.quad(lambda x: ShapedGauss(x,minuit.values[0],minuit.values[1],minuit.values[2],minuit.values[3]),
                    minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]
    ROCx2.append(1-nb/NB)
    ROCy2.append(ns/NS)
    FOM2. append(ns*np.sqrt(NB)/(NS*np.sqrt(nb)))

In [ ]:
plt.plot(ROCx2,ROCy2,label="Fit AUC = {:.1f}%".format(integrate.trapezoid(ROCy2,ROCx2)*100))
plt.plot(Brej_te, Seff_te, lw=2.5, label="Truth AUC = {:.1f}%".format(auc(Brej_te,Seff_te)*100))
plt.plot(1-fpr_gnn, tpr_gnn, lw=2.5, label="Truth AUC in peak region = {:.1f}%".format(auc(fpr_gnn,tpr_gnn)*100))
plt.xlabel(r'Background Rejection')
plt.ylabel(r'Signal Efficiency')
plt.legend()

In [ ]:
S    = [E[i] for i in range(len(y_predict)) if y_predict[i]>5/9]

chi2 = BinnedChi2(shapedgaussExp_Norm, S, bins = bins_fit , bound=fit_range)

minuit = Minuit(chi2, mu=mu, sigma=sigma, beta=beta, N=N*len(S)/len(E),  C=C*len(S)/len(E),
                    s=s, xmin=fit_range[0], xmax=fit_range[1], fix_xmin=True, fix_xmax=True)
minuit.migrad()

In [ ]:
plt.hist(Scut,50);
plt.hist(Scut[ytscut==1],50,histtype='step',lw=2.5);
a = plt.hist(Scut[ytscut==0],50,histtype='step',lw=2.5);
plt.plot(np.linspace(min(Scut),max(Scut),50),ShapedGauss(np.linspace(min(Scut),max(Scut),50),
                                                       minuit.values[0],minuit.values[1],minuit.values[2],minuit.values[3]*
                                                       (max(Scut)-min(Scut))/50))
plt.plot(np.linspace(min(Scut),max(Scut),50),expNorm(np.linspace(min(Scut),max(Scut),50),
                                                       minuit.values[4]*(max(Scut)-min(Scut))/50,
                                                     minuit.values[5],fit_range[0],fit_range[1]))
plt.plot(np.linspace(min(Scut),max(Scut),50),shapedgaussExp_Norm(np.linspace(min(Scut),max(Scut),50),
                                                       minuit.values[0],minuit.values[1],minuit.values[2],
                                                           minuit.values[3]*(max(Scut)-min(Scut))/50,
                                                           minuit.values[4]*(max(Scut)-min(Scut))/50,
                                                          minuit.values[5],minuit.values[6],minuit.values[7]))

In [ ]:
nb = integrate.quad(lambda x: expNorm(x,minuit.values[4],minuit.values[5],minuit.values[6],minuit.values[7]),
                minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]
ns = integrate.quad(lambda x: ShapedGauss(x,minuit.values[0],minuit.values[1],minuit.values[2],minuit.values[3]),
                minuit.values[0]-3*minuit.values[1],minuit.values[0]+3*minuit.values[1])[0]

In [ ]:
nb

In [ ]:
nbT

In [ ]:
ns

In [ ]:
nsT

In [ ]:
NS

In [ ]:
NST

In [ ]:
NB

In [ ]:
NBT

In [ ]:
D.RecoBig_all_10mm_R2(root='GNN_datasets/')

In [ ]:
D.RecoNew_all_10mm_R2_hitsopt_Paolina_151515(root='./')

In [ ]:
D.RecoNew_all_10mm_R2_voxopt_Paolina(root='./')